In [1]:
import os
import logging
from datetime import datetime
from mutagen.flac import FLAC
import time
import syncedlyrics
import random

In [2]:
# Configuration
MUSIC_DIR = "C:\\Users\\Max\\Desktop\\music"  # ← Change this to your actual folder
LOG_FILE = "main_log.txt"

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE, encoding='utf-8'),
        logging.StreamHandler()  # also print to console
    ]
)

def main():
    logging.info("===== Starting synced lyrics fetcher =====")
    logging.info(f"Music directory: {MUSIC_DIR}")

    if not os.path.isdir(MUSIC_DIR):
        logging.error(f"Directory not found: {MUSIC_DIR}")
        return

    flac_files = [f for f in os.listdir(MUSIC_DIR) if f.lower().endswith(".flac")] #.mp3, .wav, .aac, .m4a
    total_files = len(flac_files)
    logging.info(f"Found {total_files} FLAC files")

    processed = 0
    success = 0
    skipped = 0
    failed = 0

    for i, filename in enumerate(flac_files, 1):
        file_path = os.path.join(MUSIC_DIR, filename)
        try:
            audio = FLAC(file_path)
            artist = audio.get("artist", [""])[0].strip()
            title = audio.get("title", [""])[0].strip()
            album = audio.get("album", [""])[0].strip()

            if not (artist and title):
                logging.warning(f"[{i}/{total_files}] Skipping {filename} - missing artist or title tag")
                skipped += 1
                continue

            processed += 1
            search_query = f"{title} {artist}"
            # if album:
            #     search_query += f" {album}"

            logging.info(f"[{i}/{total_files}] Searching: {filename}")
            lrc_path = os.path.join(MUSIC_DIR, os.path.splitext(filename)[0] + ".lrc")

            try:
                # Fetch lyrics using syncedlyrics API
                # Will return synced if available, else static/unsynced automatically
                lyrics_text = syncedlyrics.search(search_query)

                if lyrics_text and len(lyrics_text.strip()) > 10:
                    with open(lrc_path, "w", encoding="utf-8") as f:
                        f.write(lyrics_text)
                    logging.info(f"SUCCESS: {filename}")
                    success += 1
                else:
                    logging.info(f"FAILURE: {filename}")
                    failed += 1

            except Exception as e_lyrics:
                logging.error(f"Error fetching lyrics for {filename}: {str(e_lyrics)}")
                failed += 1
        
            # THIS DELAY BEFORE NEW REQUEST IS VERY IMPORTANT TO AVOID GETTING FLAGGED
            delay = random.uniform(11, 15)
            logging.info(f"sleeping for - {delay:.2f} seconds")
            time.sleep(delay)

        except Exception as e_file:
            logging.error(f"Error processing {filename}: {str(e_file)}")
            failed += 1

    # Final summary
    if processed > 0:
        success_rate = (success / processed) * 100
    else:
        success_rate = 0.0

    logging.info("===== Finished processing all files =====")
    logging.info(f"Summary:")
    logging.info(f"  Total FLAC files: {total_files}")
    logging.info(f"  Processed: {processed} (skipped {skipped} due to missing tags)")
    logging.info(f"  Lyrics found: {success}/{processed} ({success_rate:.1f}% success rate)")
    logging.info(f"Log saved to: {os.path.abspath(LOG_FILE)}")    



In [3]:
main()

2026-02-01 07:49:38,289 - INFO - ===== Starting synced lyrics fetcher =====
2026-02-01 07:49:38,291 - INFO - Music directory: C:\Users\Max\Desktop\music
2026-02-01 07:49:38,306 - INFO - Found 19 FLAC files
2026-02-01 07:49:38,351 - INFO - [1/19] Searching: Abhijeet Srivastava - Sawaal.flac
2026-02-01 07:49:55,515 - ERROR - An error occurred while searching for an LRC on NetEase
2026-02-01 07:49:55,518 - ERROR - HTTPSConnectionPool(host='music.163.com', port=443): Max retries exceeded with url: /api/search/pc?limit=10&type=1&offset=0&s=Sawaal+Abhijeet+Srivastava (Caused by ConnectTimeoutError(<HTTPSConnection(host='music.163.com', port=443) at 0x1e49113a600>, 'Connection to music.163.com timed out. (connect timeout=2)'))
2026-02-01 07:50:09,102 - INFO - Found plaintext lyrics on Genius, but continuing search for synced lyrics
2026-02-01 07:50:09,119 - INFO - SUCCESS: Abhijeet Srivastava - Sawaal.flac
2026-02-01 07:50:09,119 - INFO - sleeping for - 13.49 seconds
2026-02-01 07:50:22,673 -